### Role Code Frequencies

So role code, role code source and qualified path (where in the ISO is it) for any Responsible Party, ie, contact, block.

So how many publishers in a data identification citation block?